In [ ]:
# import import_ipynb
# from dataloader import CocoDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, Normalizer
from pycocotools.cocoeval import COCOeval
import json
import torch
import numpy as np
import os

In [ ]:
def evaluate_coco(dataset, model, threshold=0.05,path=None):
    #class_names = ['AA_NA', 'AB_AP', 'AB_BI', 'AB_CA', 'AB_LI', 'EG_NA', 'LA_NA', 'PU_NA', 'QB_AP', 'QB_BI', 'QB_CA', 'QB_LI', 'disease']
    model.eval()
    
    with torch.no_grad():

        # start collecting results
        results = []
        image_ids = []
        #class_ids=[1,2,3,4,5,6,7,8,9,10,11,12,13] #개별값 뽑기 위해 필요함

        for index in range(len(dataset)):
            data = dataset[index]
            scale = data['scale']
            

            # run network
            if torch.cuda.is_available():
                scores, labels, boxes = model(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
            # else:
            #     scores, labels, boxes = model(data['img'].permute(2, 0, 1).float().unsqueeze(dim=0))
            scores = scores.cpu()
            labels = labels.cpu()
            boxes  = boxes.cpu()

            # correct boxes for image scale
            boxes /= scale

            if boxes.shape[0] > 0:
                # change to (x, y, w, h) (MS COCO standard)
                boxes[:, 2] -= boxes[:, 0]
                boxes[:, 3] -= boxes[:, 1]

                # compute predicted labels and scores
                #for box, score, label in zip(boxes[0], scores[0], labels[0]):
                for box_id in range(boxes.shape[0]):
                    score = float(scores[box_id])
                    label = int(labels[box_id])
                    box = boxes[box_id, :]

                    # scores are sorted, so we can break
                    if score < threshold:
                        break

                    # append detection for each positively labeled class
                    image_result = {
                        'image_id'    : dataset.image_ids[index],
                        'category_id' : dataset.label_to_coco_label(label),
                        'score'       : float(score),
                        'bbox'        : box.tolist(),
                    }

                    # append detection to results
                    results.append(image_result)

            # append image to list of processed images
            image_ids.append(dataset.image_ids[index])

            # print progress
            print('{}/{}'.format(index, len(dataset)), end='\r')

        if not len(results):
            return

        # write output
        json.dump(results, open('{}_bbox_results.json'.format(dataset.set_name), 'w'), indent=4)
        # load results in COCO evaluation tool
        coco_true = dataset.coco
        coco_pred = coco_true.loadRes('{}_bbox_results.json'.format(dataset.set_name))
        # run COCO evaluation
        coco_eval = COCOeval(coco_true, coco_pred, 'bbox', path=path) #실제값과 예측값에 대한 평가, 세그멘테이션 버전 변경도 가능함
        #coco_eval.params.catIds = class_ids
        coco_eval.params.maxDets=[100,200,300] #최대 detection 300까지
        coco_eval.params.imgIds = image_ids #이미지 고유아이디
        #coco_eval.params.catIds= class_ids
        coco_eval.evaluate() #예측값과 정답값을 이용하여 coco metrics를 계산함 여기서 AP, mAP, AR값을 계산함
        coco_eval.accumulate() #coco metrics를 누적함 여러개의 이미지나 클래스에 대한 metrics 평균내지않고 모두 더해놓는 것
        #coco_eval.stats()
        coco_eval.summarize() #coco metrics출력함, 개별값은 나오지 않음
        
        # 클래스별 AP
        # print("개별 클래스에 대한 AP값을 출력합니다. 해당 AP값은 IOU Threshold 0.5이상입니다.")
        # for idx, name in enumerate(class_names):
        #     print(f"{name} AP: {coco_eval.stats[0][idx+1]:.3f}") 
        model.train()

        return coco_eval